# O-H振动模之间的能量耦合，迁移和耗散

#### Date: 2025-07-09

## 目前仍需考虑的问题:
* 定位DQS哪里可以引入AQS
* 仍需要DQS有哪里是可以简化、优化的，比如对费米子哈密顿量的转换方法--**JW/BK变换**
* 怎么体现在SC平台上的实现，考虑对象是Digital/Analog block* *DQS for molecular vibration*有点“小厂”了，尽可能还是找Nature，PRL，IEEE

## 项目各阶段

### **阶段一：系统建模**
#### 时间：7.9 - 7.17

1.  **电子结构建模**
    * 用`OpenFermion`构建费米子哈密顿量，通过对其做**JW变换**(或其他变换，如**BK变换**，见*OpenFermion tutorials*) ,生成Qubit operator。
    * 为后期模拟做出准备工作--前处理 (pre-processed)。

2. **玻色子系统建模**
   * 用`Qutip`定义两个振动模（harmonic oscillator）
   * 构建能量耦合项:   $g(a_1^\dagger + a_1)(a_2^\dagger + a_2)$ 
   * 构造耦合电子态（e.g. , $σ_z(a + a^†)$ ） → 类 Holstein/JC 模型

### **阶段二：DAQC 演化结构设计**
#### 时间：7.18 - 7.25

3. **结构划分**

   * **Digital block**: 实现电子部分或耦合项。
   * **Analog block**: 模拟玻色模演化（谐振子）；
   

4. **拼接**

   * 构造 Trotter 步，拼接成总演化；
   * 用 `QuTiP` 构建 block-wise 的演化路径。


### **阶段三：耗散动力学模拟--Qutip**
#### 时间：7.26 - 7.29
#### 模拟已经有库，故可以压缩实现时间


5. **引入噪声**

   * 给两个玻色模加入 T1（amplitude damping--对应的Lindblad通道类型 ；relaxation time--能量弛豫时间）；
   * 可选：电子态加 T2（dephasing time， 退相干时间）。
   * (P.S.) T1和T2是用来描述 **量子比特退相干（decoherence）** 过程中的两种不同机制。
      * T₁ 是能量耗散时间，T₂ 是相干性丢失时间。
      * T₁ 控制“还能保持激发态多久”，
      * T₂ 控制“还能保持叠加态多久”。


6. **动力学演化**

   * 初始激发态设为 $|1⟩_1$ $|0⟩_2$；
   * 跑 `mesolve()` 得到 population, fidelity, energy transfer。
      * `qutip.mesolve()`用来数值求解 **Lindblad 主方程（master equation）**，即*模拟一个开放量子系统的时间演化*。
         * 所谓主方程（master equation）是？

            当系统不仅在演化（如由某个哈密顿量控制），**还和环境发生耗散或退相干交互**时，它的密度矩阵 $\rho(t)$ 演化不再是简单的：

            $$
            \rho(t) = U(t) \rho(0) U^\dagger(t)
            $$

            而是满足一个更复杂的方程：

            $$
            \frac{d\rho}{dt} = -i[H, \rho] + \sum_k \gamma_k \left( L_k \rho L_k^\dagger - \frac{1}{2}\{L_k^\dagger L_k, \rho\} \right)
            $$

            这就是 **Lindblad 方程**，而 `mesolve()` 就是用来解它的工具。


### **阶段四：模拟结果的对比与优化**
#### 时间：7.30 - 8.5

7. **设置多种指标(matric)，来对比DAQC和purely digital**

   * **DAQC** vs. **purely digital**（构建 purely digital circuit for reference）；
      * *Digital quantum simulation of molecular vibrations* 中模拟$H_2O$和$SO_2$的分子数各自应该都是*个位数数量* 
      --> 用于验证作者设计出来的算法
         * 另外，上述这篇文章，只涉及DQS，可以先尝试跑通 --> 复现；以及找只有AQS的文章也可以，然后将DQS 和 AQS 相结合。
         * 总之，最好是能够做出A + B的工作，而不是新建C
      
   * 评估指标
      * fidelity vs time （抗噪能力）
      * population transfer vs time
      * (P.S.)**Population transfer** 是指系统中某个量子态（或某个子空间）的**占据概率**如何随时间在各模之间转移。
         * 比如模拟两个 O–H 伸缩模（玻色模），设初始激发集中在第一个 mode：
         $$
         |\psi(0)\rangle = |1\rangle_1 \otimes |0\rangle_2
         $$
         * 随着系统演化，希望看到的是：

            * 最初 $|1⟩_1$ 模有一个振动量子；
            * 随着时间演化，这个量子（能量）**在 mode 1 和 mode 2 之间震荡、转移**；
            * 最终可能由于耗散衰减，总能量减少。
      * 模拟分子的数目
      * 模拟速度
      * 目标模拟分子的复杂程度

   * 加噪/不加噪情况对比。

### **阶段五：可选拓展&必做延申**
#### 时间： 8.6 - 8.11
8. ~~**输出至 Qiskit**~~

   ~~* 将 JW operator 转为 Qiskit 量子线路；~~
   
   ~~* 可用于硬件测试或 Aer 模拟。~~
   

9. **尝试新的技术路线**
   
   * 7.7在与谭老师的讨论中，他提到了含 $|2⟩$ 系统，即一个qubit系统包含  $|0⟩$, $|1⟩$, $|2⟩$。
      * 一般来说，用2 qubits（$|0⟩$, $|1⟩$）来模拟二能级系统；
      * 这里的含 $|2⟩$ 系统是在此基础上，添加一个$|2⟩$。通过这3个qubits去模拟二能级或是高能级系统。
      * 即**加态**，可大幅减少门深；但同时，大幅降低模拟结果的保真度。

10. **扩展至高分子（人工合成材料，或生物大分子）的分子动力学模拟**
   * **[做大分子模拟可以考虑，但短期做出成果的难度、可实现性可想而知，不过可以在 *Discussion* 部分往上凑]**
   * 基于2024年戈德贝尔奖有关AIMD的获奖论文
      * *Breaking the Million-Electron and 1 EFLOPs Barriers Biomolecular-Scale Ab Initio Molecular Dynamics*
   * 通过相关模拟，尝试、探索（高温下）高分子蛋白质的变质
   * 以及诸如蛋白质这类生物大分子的folding（折叠）现象

11. **模拟结果的（现实）应用场景**
   * 核心目标是作出更为全面的$H_2O$模拟；以及考量在哪些具体场景下，DQAS模拟的结果比传统模拟更有优势
   * 比如，在涉及到一些溶剂溶液时，简化一些条件，做**仿真模拟**